In [1]:
import logging
reload(logging)
log_fmt = '%(asctime)-9s %(levelname)-8s: %(message)s'
logging.basicConfig(format=log_fmt)

# Change to info once the notebook runs ok
#logging.getLogger().setLevel(logging.DEBUG)

In [2]:
%pylab inline

import datetime
import devlib
import os
import json
import pandas as pd
import re
import subprocess
import trappy
from trappy.plotter.Utils import get_trace_event_data

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

# Support to access the remote target
#import devlib
#from env import TestEnv

from executor import Executor

Populating the interactive namespace from numpy and matplotlib


# Target Configuration

In [3]:
# Setup a target configuration
my_target_conf = {
    
    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'juno', # Juno for now, may need to refine
    
    # Target board IP/MAC address
    "host"        : '10.0.0.57',
    
    # Login credentials
    "username"    : 'root',
    "password"    : 'insecure',

}

# Tests Configuration

In [6]:
my_tests_conf = {

    #/* Devlib modules to enable/disbale for all the experiments */
    "modules"         : [ "cpufreq" ],
    "exclude_modules" : [ "hwmon" ], # Exclude hwmon

    #/* Binary tools required by the experiments */
    "tools"    : [ "rt-app" ],

    #/* FTrace configuration */
    "ftrace" : {
        "events" : [
            "sched_switch",
            "sched_contrib_scale_f",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_boostgroup_update",
            "sched_tune_filter",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_energy_diff",
            "cpu_frequency",
            "cpu_capacity",
        ],
        "buffsize" : 10240,
    },

    #/* Set of platform configurations to test */
    "confs" : [
        {
            "tag" : "noboost",
            "flags" : "ftrace",
            "sched_features" : "ENERGY_AWARE",
            "cpufreq" : { "governor" : "sched" },
        },
    ],

    #/* Set of workloads to run on each platform configuration */
    "wloads" : {
        "mixprof" : {
            "type": "rt-app",
            "conf" : {
                "class"  : "profile",
                "params"  : {
                    "r5_10-60" : {
                        "kind"   : "Ramp",
                        "params" : {
                            "period_ms" : 16,
                            "start_pct" :  5,
                            "end_pct"   : 60,
                            "delta_pct" :  5,
                            "time_s"    :  1,
                         }
                    }
                }
            },
            "loadref" : "LITTLE",
        }
    },

    #/* Number of iterations for each workload */
    "iterations" : 1,

}

# Tests execution

In [7]:
executor = Executor(my_target_conf, my_tests_conf)

12:19:33  INFO    :         Target - Loading custom (inline) test configuration
12:19:33  INFO    :         Target - Using base path: /home/ed/ar001/lisa
12:19:33  INFO    :         Target - Loading custom (inline) target configuration
12:19:33  INFO    :         Target - Loading custom (inline) test configuration
12:19:33  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq']
12:19:33  INFO    :         Target - Connecting linux target:
12:19:33  INFO    :         Target -   username : root
12:19:33  INFO    :         Target -       host : 10.0.0.57
12:19:33  INFO    :         Target -   password : insecure
12:19:39  INFO    :         Target - Initializing target workdir:
12:19:39  INFO    :         Target -    /root/devlib-target
12:19:51  INFO    :         Target - Topology:
12:19:51  INFO    :         Target -    [[4, 5, 6, 7], [0, 1, 2, 3]]
12:19:54  INFO    :       Platform - Loading default EM:
12:19:54  INFO    :       Platform -    /home/ed/ar001/lisa/libs/utils

In [8]:
executor.run()

12:21:06  INFO    : 
12:21:06  INFO    : ################################################################################
12:21:06  INFO    :       Executor - Experiments execution
12:21:06  INFO    : ################################################################################
12:21:06  INFO    : 
12:21:06  INFO    : ================================================================================
12:21:06  INFO    :   TargetConfig - configuring target for [noboost] experiments
12:21:08  INFO    :  SchedFeatures - Set scheduler feature: ENERGY_AWARE
12:21:08  INFO    :        CPUFreq - Configuring all CPUs to use [sched] governor
12:21:09  INFO    :          WlGen - Setup new workload mixprof
12:21:09  INFO    :          RTApp - Workload duration defined by longest task
12:21:09  INFO    :          RTApp - Default policy: SCHED_OTHER
12:21:09  INFO    :          RTApp - ------------------------
12:21:09  INFO    :          RTApp - task [task_r5_10-60], sched: using default policy
12

In [9]:
!tree {executor.te.res_dir}

/home/ed/ar001/lisa/results/20160608_122044
└── rtapp:noboost:mixprof
    ├── 1
    │   ├── mixprof_00.json
    │   ├── output.log
    │   ├── rt-app-task_r5_10-60-0.log
    │   └── trace.dat
    ├── kernel.config
    ├── kernel.version
    └── platform.json

2 directories, 7 files
